In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [3]:
from src.models.bidir_lstm_multihead import build_model

In [4]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/bidir_lstm/1k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/bidir_lstm/1k', 'model': {'type': 'Bidirectional Multihead LSTM', 'lstm_units': 128, 'num_lstm_layers': 2, 'attention': True}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'path': '/games/tic-tac-toe/1k_single_agent.csv'}}


In [6]:
model_params = {
    'lstm_units': config['model']['lstm_units'],
    'num_lstm_layers': config['model']['num_lstm_layers'],
    'attention': config['model']['attention']
}

In [7]:
# Load and preprocess data
df = load_data(f'../../../../data/processed/{config['data']['path']}')
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model,
)

Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - loss: 4.1058 - output_1_accuracy: 0.8092 - output_2_accuracy: 0.4181 - output_3_accuracy: 0.1268 - val_loss: 3.0492 - val_output_1_accuracy: 0.9026 - val_output_2_accuracy: 0.5964 - val_output_3_accuracy: 0.2130
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 3.0377 - output_1_accuracy: 0.8922 - output_2_accuracy: 0.5849 - output_3_accuracy: 0.2415 - val_loss: 2.8676 - val_output_1_accuracy: 0.9125 - val_output_2_accuracy: 0.7000 - val_output_3_accuracy: 0.2146
Epoch 3/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 2.8172 - output_1_accuracy: 0.9229 - output_2_accuracy: 0.6834 - output_3_accuracy: 0.2457 - val_loss: 2.6469 - val_output_1_accuracy: 0.8964 - val_output_2_accuracy: 0.8979 - val_output_3_accuracy: 0.2323
Epoch 4/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 2.3958 - output_1_accuracy: 0.9175 - output_2_accuracy: 0.9193 - output_3_accuracy: 0.2506 - val_loss: 2.2831 - val_output_1_accuracy: 0.9312

Test Results:
loss: 2.2548
compile_metrics: 0.9312


In [9]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 527ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Initial sequence:
MOVE, X, 0,0
MOVE, O, 1,1
MOVE, X, 2,0
MOVE, O, 1,2
MOVE, X, 0,1
MOVE, O, 0,2
MOVE, X, 2,2
MOVE, O, 1,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 0,0
MOVE, O, 0,1
MOVE, X, 2,1
MOVE, O, 1,0
MOVE, X, 1,1
MOVE, O, 0,2
MOVE, X, 2,0
MOVE, O, 1,2
MOVE, X, 2,2
GAME_END, system, X

Predicted moves:
Move 1: GAME_START, system, New Game
Move 2: MOVE, X, 2,2
Move 3: MOVE, O, 0,1
Move 4: MOVE, X, 0,1
Mov

In [10]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f'../../../../models/{config['name']}.h5',
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Initial sequence:
MOVE, O, 2,2
MOVE, X, 2,1
MOVE, O, 0,1
MOVE, X, 0,0
MOVE, O, 2,0
MOVE, X, 1,0
GAME_END, system, draw
GAME_START, system, New Game
MOVE, X, 2,0
MOVE, O, 1,2
MOVE, X, 0,2
MOVE, O, 2,1
MOVE, X, 2,2
MOVE, O, 1,0
MOVE, X, 0,1
MOVE, O, 0,0
MOVE, X, 1,1
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 1,1

Predicted moves:
Move 1: MOVE, O, 1,1
Move 2: MOVE, X, 2,2
Move 3: MOVE, O, 1,0
Move 4: MOVE, X, 0,1
